In [ ]:
import MetaTrader5 as mt
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from technical_analysis import moving_average
from technical_analysis.backtest import Backtest
from technical_analysis.backtest.strategy import MovingAverageCrossover
from technical_analysis import overlays
import time
plt.style.use("seaborn-v0_8")

In [ ]:
mt.initialize()

In [ ]:
login = *********
password = '#######'
server = 'Exness-MT5Trial7'

In [ ]:
mt.login(login,password,server)

In [ ]:
ticker = 'XAUUSDm'
qty = 0.01
buy_order_type = mt.ORDER_TYPE_BUY
sell_order_type = mt.ORDER_TYPE_SELL
buy_price = mt.symbol_info_tick(ticker).bid
sell_price = mt.symbol_info_tick(ticker).ask

In [ ]:
ticker = 'XAUUSDm'
qty = 1.00
buy_order_type = mt.ORDER_TYPE_BUY
sell_order_type = mt.ORDER_TYPE_SELL
buy_price = mt.symbol_info_tick(ticker).bid
sell_price = mt.symbol_info_tick(ticker).ask
buy_sl = buy_price - 1.200
buy_tp = buy_price + 6.300
sell_sl = sell_price + 1.200
sell_tp = sell_price - 6.300

def create_buy_order() :
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": 'XAUUSDm',
        "volume": 0.01,
        "type": mt.ORDER_TYPE_BUY,
        "price" : mt.symbol_info_tick(ticker).bid,
        "sl": mt.symbol_info_tick(ticker).bid - 1.300,
        "tp": mt.symbol_info_tick(ticker).bid + 5.200,
        "comment": "Python open Position",
        "type_time":mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC
    }
    # send a trading request
    order = mt.order_send(request)
    return order


def create_sell_order() :
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": 'XAUUSDm',
        "volume": 0.01,
        "type": mt.ORDER_TYPE_SELL,
        "price" : mt.symbol_info_tick(ticker).ask,
        "sl": mt.symbol_info_tick(ticker).ask + 1.300,
        "tp": mt.symbol_info_tick(ticker).ask - 5.200,
        "comment": "Python open Position",
        "type_time":mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC
    }
    # send a trading request
    order = mt.order_send(request)
    return order


def close_buy_order() :
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": 'XAUUSDm',
        "volume": 0.01,
        "type": mt.ORDER_TYPE_BUY,
        "position": mt.positions_get()[0]._asdict()["ticket"],
        "price" :  mt.symbol_info_tick(ticker).bid,
        "comment": "Close Position",
        "type_time":mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC
    }
    result = mt.order_send(request)


def close_sell_order() :
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": 'XAUUSDm',
        "volume": 0.01,
        "type": mt.ORDER_TYPE_SELL,
        "position": mt.positions_get()[0]._asdict()["ticket"],
        "price" : mt.symbol_info_tick(ticker).ask,
        "comment": "Close Position",
        "type_time":mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC
    }
    result = mt.order_send(request)
    
    # send a trading request
    result = mt.order_send(request)


In [ ]:
count = 0
account_info = mt.account_info()
account_balance = account_info.balance
loss_target = account_balance - 7.500
print(account_balance)
print(loss_target)

In [ ]:
for i in range(200) :
    account_info = mt.account_info()
    account_balance = account_info.balance
    
    if account_balance >= loss_target :

        ohlc = pd.DataFrame(mt.copy_rates_range('XAUUSDm',mt.TIMEFRAME_M1,datetime(2024,2,15),datetime.now()))
        ohlc['time'] = pd.to_datetime(ohlc['time'],unit='s')

        ohlc["ema_s"] = moving_average.ema(ohlc.close, period=1)
        ohlc["ema_l"] = moving_average.ema(ohlc.close, period=2)

        ohlc["position"] = np.where(ohlc["ema_s"] > ohlc["ema_l"], 1, -1 )
        ohlc.dropna(inplace = True)
        data = ohlc.iloc[-1]


        # if mt.positions_get()[0]._asdict()["symbol"] != 'XAUUSDm' :
        #     #if mt.positions_get()[-1]._asdict()["ticket"]
                    
        if len(mt.positions_get()) == 1 :
            if mt.positions_get()[0]._asdict()["type"] == 0 :
                if data.position == -1 :
                    close_buy_order()
                    print("Buy order closed")
            if mt.positions_get()[0]._asdict()["type"] == 1 :
                if data.position == +1 :
                    close_sell_order()
                    print("Sell order closed")
 

        if len(mt.positions_get()) == 0 :
            if data.position == 1 :
                #if data.close > data.ema_s :
                create_buy_order()
                print("Buy order placed")
            elif data.position == -1 :
                #if data.close < data.ema_s :
                create_sell_order()
                print("Sell order placed")
            

        count += 1    
        print(ohlc)
        print(count)
        print(account_balance)
        print(loss_target)
        time.sleep(60)